# Тестовые примеры для проверки работы pytorch

In [7]:
import torch

# Проверка доступности GPU
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available")

# Тестовая операция на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.rand(3, 3).to(device)  # Создаём случайный тензор и отправляем на устройство
print(f"Tensor created on: {x.device}")

GPU is available: NVIDIA GeForce RTX 3080 Ti
Tensor created on: cuda:0


In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time

# Настройки
batch_size = 64
num_epochs = 1  # Уменьшено для быстрой проверки

# Загрузка данных
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Определение модели
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Преобразуем изображения в векторы
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Функция для обучения модели
def train_model(device):
    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    start_time = time.time()  # Записываем время начала

    for epoch in range(num_epochs):
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()   # Обнуляем градиенты
            outputs = model(images) # Прямой проход
            loss = criterion(outputs, labels) # Вычисляем потерю
            loss.backward()         # Обратный проход
            optimizer.step()        # Обновляем параметры

    end_time = time.time()  # Записываем время окончания
    return end_time - start_time  # Возвращаем время выполнения

# Обучение на CPU
print("Training on CPU...")
cpu_time = train_model(device='cpu')
print(f"Training time on CPU: {cpu_time:.4f} seconds")

# Обучение на GPU
print("Training on GPU...")
gpu_time = train_model(device='cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training time on GPU: {gpu_time:.4f} seconds")


Training on CPU...
Training time on CPU: 18.5004 seconds
Training on GPU...
Training time on GPU: 19.3624 seconds


# Проверки

In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создаем случайный тензор и перемещаем его на GPU
x = torch.randn(10000, 10000).to(device)

# Выполняем простую операцию, чтобы загрузить GPU
y = torch.mm(x, x)

In [10]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Должно вернуть True
print(torch.cuda.current_device())  # Должно вернуть ID текущего устройства
print(torch.cuda.get_device_name(0))  # Имя вашего GPU

2.4.1
True
0
NVIDIA GeForce RTX 3080 Ti


In [12]:
import torch
import time

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Функция для выполнения вычислений на GPU
def gpu_load_test():
    # Количество итераций
    iterations = 100
    # Размер тензоров
    tensor_size = (10000, 10000)

    for i in range(iterations):
        # Создание случайного тензора
        x = torch.rand(tensor_size, device=device)
        y = torch.rand(tensor_size, device=device)

        # Выполнение операций
        z = torch.matmul(x, y)

        # Ожидание, чтобы не перегружать GPU слишком быстро
        time.sleep(0.1)

        # Освобождение памяти
        del x, y, z


# Запуск теста
gpu_load_test()

In [15]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Проверка на наличие GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

# 1. Подготовка данных
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Используем набор CIFAR-10 для теста
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2. Определение простой модели
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Инициализация модели
model = SimpleCNN().to(device)

# 3. Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Функция обучения модели
def train_model(model, train_loader, criterion, optimizer, epochs=5, device=device):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# 5. Функция для оценки модели
def evaluate_model(model, test_loader, device=device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Точность на тестовых данных: {accuracy:.2f}%")

# Обучение на GPU
start_time = time.time()
print("\nОбучение на GPU...")
train_model(model, train_loader, criterion, optimizer, epochs=5)
gpu_time = time.time() - start_time
print(f"Обучение завершено за {gpu_time:.2f} секунд")

# Оценка модели на GPU
evaluate_model(model, test_loader)

# Перенос модели на CPU
model_cpu = SimpleCNN().to('cpu')

# Переинициализация оптимизатора для CPU
optimizer_cpu = optim.Adam(model_cpu.parameters(), lr=0.001)

# Обучение на CPU
start_time = time.time()
print("\nОбучение на CPU...")
train_model(model_cpu, train_loader, criterion, optimizer_cpu, epochs=5, device='cpu')
cpu_time = time.time() - start_time
print(f"Обучение завершено за {cpu_time:.2f} секунд")

# Оценка модели на CPU
evaluate_model(model_cpu, test_loader, device='cpu')


Используемое устройство: cuda
Files already downloaded and verified
Files already downloaded and verified

Обучение на GPU...
Epoch 1/5, Loss: 1.4156
Epoch 2/5, Loss: 1.0285
Epoch 3/5, Loss: 0.8503
Epoch 4/5, Loss: 0.7141
Epoch 5/5, Loss: 0.5901
Обучение завершено за 99.55 секунд
Точность на тестовых данных: 70.94%

Обучение на CPU...
Epoch 1/5, Loss: 1.4511
Epoch 2/5, Loss: 1.0594
Epoch 3/5, Loss: 0.8921
Epoch 4/5, Loss: 0.7655



KeyboardInterrupt



In [1]:
import torch

# Check what version of PyTorch is installed
print(torch.__version__)

# Check the current CUDA version being used
print("CUDA Version: ", torch.version.cuda)

# Check if CUDA is available and if so, print the device name
print("Device name:", torch.cuda.get_device_properties("cuda").name)

# Check if FlashAttention is available
print("FlashAttention available:", torch.backends.cuda.flash_sdp_enabled())

2.4.1
CUDA Version:  11.8
Device name: NVIDIA GeForce RTX 3080 Ti
FlashAttention available: True
